# Budgeted MAB Algorithms

In [1]:
%matplotlib inline
import scipy.stats as st
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import math
import numpy as np
import pandas as pd
import copy
import sys
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
plt.rcParams['font.size'] = 13
plt.style.use("ggplot")
mpl.rcParams['font.family'] = "Osaka"

## Common framework

Consider the case that there are 3 arms: $A = \{a_1, a_2, a_3\}$. The total budget is given by $B(\geq 0)$. At each round $t = 1, 2, \ldots$, a player chooses an arm $a_t \in A$ and gets a random reward $r_t$ and pays a random cost $c_t$. We assume rewards and costs are i.i.d. sample from the bernouli distribution $Ber(p_a), Ber(c_a)$, where 

$$
(p_a, c_a) = \begin{cases}
(0.1, 0.4) & \text{if $a = a_1$} \\
(0.5, 0.5) & \text{if $a = a_2$} \\
(0.6, 0.8) & \text{if $a = a_3$}
\end{cases}, \hspace{1em} \frac{p_a}{c_a} = \begin{cases}
0.25 & \text{if $a = a_1$} \\
1.0 & \text{if $a = a_2$} \\
0.75 & \text{if $a = a_3$}
\end{cases}
$$

A player continues to pick arms until the budget runs out.

In [2]:
class BMAB(object):
    def __init__(self, n_arms, payoff_dists, cost_dists, rs=None):
        self.n_arms = n_arms
        self.payoff_dists = payoff_dists
        self.cost_dists = cost_dists
        if rs is None:
            self.seed = np.random.randint(2**32 - 1)
            self.rs = np.random.RandomState(self.seed)
        else:
            self.rs = rs

    def pull_arm(self, arm):
        if not isinstance(arm, int) or arm < 0 or self.n_arms <= arm:
            raise ValueError(arm)
        
        payoff = np.sum(self.payoff_dists[arm].rvs(1, random_state=self.rs))
        cost = np.sum(self.cost_dists[arm].rvs(1, random_state=self.rs))
        return payoff, cost
    

In [3]:
n_arms = 3
expected_payoffs = [0.1, 0.5, 0.1]
expected_costs = [0.4, 0.5, 0.8]

payoff_dists = [st.bernoulli(p) for p in expected_payoffs]
cost_dists = [st.bernoulli(p) for p in expected_costs]

bmab = BMAB(n_arms, payoff_dists, cost_dists)

for i in range(30):
    arm = i // 10
    p, c = bmab.pull_arm(arm)
    print("t={0:d}, arm={1:d}, reward={2: .3f}, cost={3: .3f}".format(i, arm, p, c))

t=0, arm=0, reward= 0.000, cost= 0.000
t=1, arm=0, reward= 0.000, cost= 0.000
t=2, arm=0, reward= 1.000, cost= 0.000
t=3, arm=0, reward= 0.000, cost= 0.000
t=4, arm=0, reward= 0.000, cost= 0.000
t=5, arm=0, reward= 0.000, cost= 0.000
t=6, arm=0, reward= 0.000, cost= 1.000
t=7, arm=0, reward= 0.000, cost= 0.000
t=8, arm=0, reward= 1.000, cost= 0.000
t=9, arm=0, reward= 0.000, cost= 1.000
t=10, arm=1, reward= 0.000, cost= 1.000
t=11, arm=1, reward= 1.000, cost= 0.000
t=12, arm=1, reward= 0.000, cost= 1.000
t=13, arm=1, reward= 0.000, cost= 1.000
t=14, arm=1, reward= 0.000, cost= 0.000
t=15, arm=1, reward= 0.000, cost= 1.000
t=16, arm=1, reward= 1.000, cost= 1.000
t=17, arm=1, reward= 1.000, cost= 0.000
t=18, arm=1, reward= 1.000, cost= 0.000
t=19, arm=1, reward= 0.000, cost= 0.000
t=20, arm=2, reward= 0.000, cost= 1.000
t=21, arm=2, reward= 0.000, cost= 0.000
t=22, arm=2, reward= 0.000, cost= 1.000
t=23, arm=2, reward= 0.000, cost= 1.000
t=24, arm=2, reward= 0.000, cost= 1.000
t=25, arm=

In [4]:
class Policy(object):
    def __init__(self, bmab, budget, rs=None):
        self.bmab = bmab
        self.budget = budget
        
        if rs is None:
            self.seed = np.random.randint(2**32 - 1)
            self.rs = np.random.RandomState(self.seed)
        else:
            self.rs = rs
        
        self.n_arms = self.bmab.n_arms
        self.t = 0
        self.n_pull = np.zeros(self.n_arms, dtype=int)
        self.sum_payoffs = np.zeros(self.n_arms, dtype=float)
        self.sum_costs = np.zeros(self.n_arms, dtype=float)
        
    def proceed(self):
        pass
    
    def get_average_score(self):
        average_payoff = np.sum(self.sum_payoffs) / self.t
        average_cost = np.sum(self.sum_costs) / self.t
        return average_payoff, average_cost
        

In [5]:
seed = 123

## 1. Budgeted Thompson Sampling

In [6]:
class BudgetedThompsonSampling(Policy):
    """
    Implement the Thompson-Sampling policy.
    
    * payoff_priors: prior distributions of the rewards of arms
    * cost_priors: prior distributions of the costs of arms
    * payoff_update_func: function that updates payoff prior distributions
    * cost_update_func: function that updates cost prior distributions
    """
    def __init__(self, bmab, budget, payoff_priors, cost_priors, payoff_update_func, cost_update_func, rs=None):
        super().__init__(bmab, budget, rs)
        
        if len(payoff_priors) != bmab.n_arms or len(cost_priors) != bmab.n_arms:
            raise ValueError
        
        self.payoff_priors = copy.deepcopy(payoff_priors)
        self.cost_priors = copy.deepcopy(cost_priors)
        self.payoff_update_func = payoff_update_func
        self.cost_update_func = cost_update_func
        
    def update_priors(self, arm, payoff, cost):
        self.payoff_priors = self.payoff_update_func(self.payoff_priors, arm, payoff)
        self.cost_priors = self.cost_update_func(self.cost_priors, arm, cost)
        
    def proceed(self):
        if self.budget <= 0:
            raise ValueError("Budget runs out")
            
        # choose an arm
        rates = np.empty(self.n_arms)
        for a in range(self.n_arms):
            p = self.payoff_priors[a].rvs(random_state=self.rs)
            c = self.cost_priors[a].rvs(random_state=self.rs)
            rates[a] = p / c
        
        arm = int(np.argmax(rates))
        
        p, c = self.bmab.pull_arm(arm)
        self.n_pull[arm] += 1
        self.sum_payoffs[arm] += p
        self.sum_costs[arm] += c
        self.update_priors(arm, p, c)
        self.budget -= c
        self.t += 1
        

For consistency, non-optimal arms should be pulled at least $O(\log t)$ times by $t$ period

In [7]:
budget = 100
payoff_priors = [st.beta(1, 1) for i in range(n_arms)]
cost_priors = [st.beta(1, 1) for i in range(n_arms)]
max_expected_payoff_per_cost = 1.0
rs = np.random.RandomState(seed)
bmab = BMAB(n_arms, payoff_dists, cost_dists, rs=rs)

def update_priors(priors, arm, p):
    if p == 1:
        a = priors[arm].args[0] + 1
        b = priors[arm].args[1]
    elif p == 0:
        a = priors[arm].args[0]
        b = priors[arm].args[1] + 1
    else:
        raise ValueError
    
    priors[arm].args = (a, b)
    return priors


thompson = BudgetedThompsonSampling(
    bmab, budget, payoff_priors, cost_priors, update_priors, update_priors, rs=rs)

while True:
    if thompson.budget <= 0:
        break
    
    thompson.proceed()
    p, c = thompson.get_average_score()
    
    if thompson.t % 10 == 1:
        print(
            "t: {}, n_arms: {}, mean_payoff: {:.3f}, mean_cost: {:.3f}"
            .format(thompson.t, thompson.n_pull, p, c)
        )
        

t: 1, n_arms: [1 0 0], mean_payoff: 0.000, mean_cost: 1.000
t: 11, n_arms: [2 4 5], mean_payoff: 0.091, mean_cost: 0.818
t: 21, n_arms: [5 7 9], mean_payoff: 0.095, mean_cost: 0.810
t: 31, n_arms: [12  9 10], mean_payoff: 0.129, mean_cost: 0.742
t: 41, n_arms: [13 16 12], mean_payoff: 0.146, mean_cost: 0.659
t: 51, n_arms: [14 24 13], mean_payoff: 0.216, mean_cost: 0.647
t: 61, n_arms: [14 33 14], mean_payoff: 0.262, mean_cost: 0.656
t: 71, n_arms: [14 43 14], mean_payoff: 0.324, mean_cost: 0.620
t: 81, n_arms: [15 52 14], mean_payoff: 0.321, mean_cost: 0.568
t: 91, n_arms: [15 62 14], mean_payoff: 0.352, mean_cost: 0.571
t: 101, n_arms: [15 72 14], mean_payoff: 0.386, mean_cost: 0.554
t: 111, n_arms: [15 82 14], mean_payoff: 0.405, mean_cost: 0.550
t: 121, n_arms: [15 92 14], mean_payoff: 0.421, mean_cost: 0.537
t: 131, n_arms: [ 15 102  14], mean_payoff: 0.427, mean_cost: 0.534
t: 141, n_arms: [ 15 112  14], mean_payoff: 0.433, mean_cost: 0.539
t: 151, n_arms: [ 15 122  14], mean_pay

In [14]:
budget = 500
size = 20
rs = np.random.RandomState(seed)
bmab = BMAB(n_arms, payoff_dists, cost_dists, rs=rs)

rewards = np.zeros(size)
ts_lengths = np.zeros(size)

for i in range(size):
    budgeted_thompson = BudgetedThompsonSampling(
        bmab, budget, payoff_priors, cost_priors, update_priors, update_priors, rs=rs)
    
    while budgeted_thompson.budget > 0:
        budgeted_thompson.proceed()

    p, c = budgeted_thompson.get_average_score()
    rewards[i] = p
    ts_lengths[i] = budgeted_thompson.t

mean_rewards = np.mean(rewards * ts_lengths)
mean_regrets = budget - mean_rewards # optimal arm: payoff 0.5 per cost 0.5
mean_ts_length = np.mean(ts_lengths)

In [15]:
upper_bound_regret = 0
for a in range(n_arms):
    r = expected_payoffs[a] / expected_costs[a]
    diff = 1.0 - r
    
    # if not optimal arm
    if np.abs(diff) > 1e-3:
        d = expected_costs[a] * diff # gamma = 1
        upper_bound_regret += 8 * np.log(budget) / d

print(mean_rewards, mean_regrets, upper_bound_regret, mean_ts_length)

484.8 15.2 236.746975178 1009.75
